In [ ]:
import datetime as dt
import pandas as pd
import numpy as np

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from validator_collection import validators, checkers
from urllib.request import urlopen
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
from newspaper import Article
import scrapy
import requests
import time

import nltk
nltk.download('punkt')

In [ ]:
def get_news_class_hrefs(url):
    """
    Finds all urls pointed to by all links inside
    'news' class div elements
    """
    #https://stackoverflow.com/questions/36709165/beautifulsoup-object-of-type-response-has-no-len
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    #https://stackoverflow.com/questions/32468801/how-to-get-href-from-an-a-tag-inside-a-div
    links = [a['href'] for div in soup.find_all("div", attrs={"class": "blog-list-blog"}) for a in div.find_all('a')]
    return list(dict.fromkeys(links))


In [ ]:
InvalidURL = list()
def get_short_news(url):
    if checkers.is_url(url):
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html, "html.parser")
        short_news = soup.find("div", attrs={"class": "article-bnow-box"}).h2.text
    else:
        short_news = "Invalid URL given"
        InvalidURL.append(url)
        
    return short_news

In [ ]:
UnProcessedUrls = list()
def get_news_details(url):

    try:
        toi_article = Article(url, language="en")
        toi_article.download()
        toi_article.parse()
        time.sleep(3)
        toi_article.nlp()
    
        ArticleTitle = toi_article.title
        ArticleText = toi_article.text
        ArticleSummary = toi_article.summary
        ArtPubDate = toi_article.publish_date
        return ArticleTitle, ArticleText, ArticleSummary, ArtPubDate

    except:
        UnProcessedUrls.append(url)
        return "No Text", "No Text", "No Text", "No Text"
    

In [ ]:
News18_Articles = pd.DataFrame(columns = ['Date', 'Title', 'Text', 'Summary', 'Link'])
invalid_url, ArtTitle, ArtTest, ArtSummary, ArtDate, processedURLs = ([] for i in range(6))
for i in range(72,100):
    print('Number of Pgs Completed = ', i)
    #https://www.news18.com/politics/page-1/
    pageUrl = 'https://www.news18.com/politics/page-'+'{}/'.format(i)
    processedURLs.append(pageUrl)
    ArtLinks = get_news_class_hrefs(pageUrl)
    
    #https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests
    for artLink in ArtLinks:
        if checkers.is_url(artLink):
            if not None:
                Title, Text, Summary, Date = get_news_details(artLink)
                #short_news = get_short_news(artLink)
                temp_df = pd.DataFrame({'Date' : Date,
                                        'Title' : Title,
                                        'Text' : Text,
                                        'Summary' : Summary,
                                        'Link': artLink}, index=[0])
                News18_Articles = News18_Articles.append(temp_df, ignore_index=True, sort=True)
                ArtTitle.append(Title)
                ArtTest.append(Text)
                ArtSummary.append(Summary)
                ArtDate.append(Date)
                #Short_News.append(short_news)
            
        else:
            invalid_url.append(artLink)
    print('Total News Articles Scrapped =', len(ArtTitle))

In [ ]:
News18_Articles.tail()

In [ ]:
len(News18_Articles)

In [ ]:
News18_Articles.drop_duplicates(subset = 'Title', keep = 'first', inplace = True)

In [ ]:
len(News18_Articles)

In [ ]:
News18_Articles.to_csv(r'C:\Users\shrikanth\Desktop\University of Passau\Text Mining Project\Project Implementation\CSV Files\News18\News18_Articles_II.csv', 
            index=False, header=True)